In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import plotly.express as px

# Load datasets
df_confirmed_cases = pd.read_csv("time_series_covid19_confirmed_US.csv")
df_deaths = pd.read_csv("time_series_covid19_deaths_US.csv")
df_vaccinations = pd.read_csv("us_state_vaccinations.csv")

# Preprocess data
# For geospatial visualization, we need to aggregate the confirmed cases by state
df_confirmed_by_state = df_confirmed_cases.groupby("Province_State").sum().reset_index()

# Calculate population trend for death cases using the "Population" column from df_deaths
# Handle non-numeric values when summing the columns
df_deaths_numeric = df_deaths.drop(columns=["UID", "iso2", "iso3", "code3", "FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Combined_Key"])
df_deaths_numeric = df_deaths_numeric.apply(pd.to_numeric, errors='coerce')
df_deaths_numeric = df_deaths_numeric.fillna(0)  # Replace NaNs with 0
df_confirmed_numeric = df_confirmed_cases.drop(columns=["UID", "iso2", "iso3", "code3", "FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Combined_Key"])
df_confirmed_numeric = df_confirmed_numeric.apply(pd.to_numeric, errors='coerce')
df_confirmed_numeric = df_confirmed_numeric.fillna(0)  # Replace NaNs with 0
df_deaths["Population_trend"] = df_deaths["Population"] * (df_deaths_numeric.sum(axis=1) / df_confirmed_numeric.sum(axis=1))

# Create a Folium map
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add markers to the map with radius based on confirmed cases
marker_cluster = MarkerCluster().add_to(m)
for idx, row in df_confirmed_by_state.iterrows():
    state = row["Province_State"]
    cases = row[df_confirmed_cases.columns[11:]].sum()  # Summing confirmed cases from date columns
    radius = min(cases / 10000, 50)  # Scale radius to fit within reasonable bounds
    folium.CircleMarker(location=[row["Lat"], row["Long_"]], radius=radius, color='blue', fill=True).add_to(marker_cluster)

# Color states based on total number of cases
choropleth = folium.Choropleth(
    geo_data='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json',
    data=df_confirmed_by_state,
    columns=['Province_State', df_confirmed_cases.columns[11]],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Confirmed Cases'
).add_to(m)

# Create line plot for Date vs Number of Confirmed Cases using Plotly
dates = df_confirmed_cases.columns[11:]
confirmed_cases = df_confirmed_cases[dates].sum()
fig = px.line(x=dates, y=confirmed_cases, title='Date vs Number of Confirmed Cases')
fig.update_xaxes(title='Date', tickangle=90)
fig.update_yaxes(title='Number of Confirmed Cases')
fig.show()

# Create line plot for Population Trend for Death Cases using Plotly
fig = px.line(x=df_deaths.index, y=df_deaths["Population_trend"], title='Population Trend for Death Cases')
fig.update_xaxes(title='Date', tickangle=90)
fig.update_yaxes(title='Population Trend for Death Cases')
fig.show()

